<a href="https://colab.research.google.com/github/ayush-kaurav0512/Sberbank-Russian-Housing-Market/blob/main/Sberbank_Russian_Housing_Market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import all important packages and library
!pip install scikit-learn
!pip install vecstack
import pandas as pd
import numpy as np
import sklearn as sk
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.svm import LinearSVR
from vecstack import stacking
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error, r2_score




  Preparing metadata (setup.py) ... done
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19861 sha256=662f77747cfd3437f4227f2c97d2559daf534067bf8bfbaaf06495661ca84821
  Stored in directory: /root/.cache/pip/wheels/b8/d8/51/3cf39adf22c522b0a91dc2208db4e9de4d2d9d171683596220
Successfully built vecstack
Mounted at /content/drive


In [ ]:
# read the test data

train_data=pd.read_csv('/content/drive/MyDrive/machine_learning_dataset/train.csv')
train_data.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [ ]:
# And check for null value in what columns

print(train_data.isnull().sum())
train_data['timestamp']=pd.to_datetime(train_data['timestamp'])
# train_data.set_index('timestamp',inplace=True)

id                       0
timestamp                0
full_sq                  0
life_sq               6383
floor                  167
                      ... 
mosque_count_5000        0
leisure_count_5000       0
sport_count_5000         0
market_count_5000        0
price_doc                0
Length: 292, dtype: int64


In [ ]:
# Handle missing values in the training dataset

train_data.interpolate(method='linear',inplace=True,)
train_data.isnull().sum()

<ipython-input-4-773292540423>:3: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  train_data.interpolate(method='linear',inplace=True,)


,0
id,0
timestamp,0
full_sq,0
life_sq,0
floor,0
...,...
mosque_count_5000,0
leisure_count_5000,0
sport_count_5000,0
market_count_5000,0


In [ ]:
train_data.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [ ]:
# prompt: still null values are there after applying interpolate  only

# Handle missing values in the training dataset using forward fill for remaining nulls
train_data['build_year'].interpolate(method='linear', inplace=True)

# Check for null values again
print(train_data.isnull().sum())

id                    0
timestamp             0
full_sq               0
life_sq               0
floor                 0
                     ..
mosque_count_5000     0
leisure_count_5000    0
sport_count_5000      0
market_count_5000     0
price_doc             0
Length: 292, dtype: int64


<ipython-input-6-d1c9fb0d3ce2>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['build_year'].interpolate(method='linear', inplace=True)


In [ ]:
train_data.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [ ]:
print(train_data.isnull().sum())

id                    0
timestamp             0
full_sq               0
life_sq               0
floor                 0
                     ..
mosque_count_5000     0
leisure_count_5000    0
sport_count_5000      0
market_count_5000     0
price_doc             0
Length: 292, dtype: int64


In [ ]:
train_data.dropna(inplace=True)

train_data.isnull().sum()

,0
id,0
timestamp,0
full_sq,0
life_sq,0
floor,0
...,...
mosque_count_5000,0
leisure_count_5000,0
sport_count_5000,0
market_count_5000,0


In [ ]:
train_data.reset_index(drop=True,inplace=True)
train_data.head()

# train_data.dropna(inplace=True, axis=1)

if train_data.isnull().any().any():
    print("There is at least one null value in the DataFrame.")
else:
    print("There are no null values in the DataFrame.")

There are no null values in the DataFrame.


In [ ]:
# now Merge the train_data with Macroeconomics

macro_data=pd.read_csv('/content/drive/MyDrive/machine_learning_dataset/macro.csv')
macro_data.head()

# change to data time
macro_data['timestamp']=pd.to_datetime(macro_data['timestamp'])
macro_data.head()

,timestamp,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_deflator,balance_trade,balance_trade_growth,usdrub,...,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
0,2010-01-01,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
1,2010-01-02,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
2,2010-01-03,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
3,2010-01-04,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.905,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN
4,2010-01-05,76.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.836,...,690.0,6221.0,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN


In [ ]:
from typing_extensions import final
#perform the join

final_train_df=pd.merge(train_data, macro_data,on='timestamp', how='left')
final_train_df.tail()

# Check if price_doc is present or not
if 'price_doc' in final_train_df.columns:
  print('price_doc is present')
else:
  print('price_doc is not present')

# move the price_doc to the last position
price_doc_col=final_train_df.pop('price_doc')
final_train_df['price_doc']=price_doc_col
final_train_df.head()

price_doc is present


,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm,price_doc
0,8059,2013-05-21,11,11.0,2.0,5.000000,2.000000,1907.000000,1.000000,12.000000,...,9350.0,627.0,0.43939,1440.0,329348.0,26.7,76.2,46352.0,226047.3,2750000
1,8060,2013-05-22,45,11.5,1.0,5.309091,2.036364,1907.924051,1.036364,11.781818,...,9350.0,627.0,0.43939,1440.0,329348.0,26.7,76.2,46352.0,226047.3,4011920
2,8061,2013-05-22,68,12.0,2.0,5.618182,2.072727,1908.848101,1.072727,11.563636,...,9350.0,627.0,0.43939,1440.0,329348.0,26.7,76.2,46352.0,226047.3,5406690
3,8062,2013-05-22,68,12.5,2.0,5.927273,2.109091,1909.772152,1.109091,11.345455,...,9350.0,627.0,0.43939,1440.0,329348.0,26.7,76.2,46352.0,226047.3,5406690
4,8063,2013-05-22,21,13.0,7.0,6.236364,2.145455,1910.696203,1.145455,11.127273,...,9350.0,627.0,0.43939,1440.0,329348.0,26.7,76.2,46352.0,226047.3,4500000


In [ ]:
# check for null valus in final_train_df

print(final_train_df.isnull().sum())

id                                 0
timestamp                          0
full_sq                            0
life_sq                            0
floor                              0
                               ...  
population_reg_sports_share     3239
students_reg_sports_share      16901
apartment_build                 3239
apartment_fund_sqm                 0
price_doc                          0
Length: 391, dtype: int64


In [ ]:
# Handlr missing value

final_train_df.interpolate(method='linear',inplace=True)
print(final_train_df.isnull().sum())

id                             0
timestamp                      0
full_sq                        0
life_sq                        0
floor                          0
                              ..
population_reg_sports_share    0
students_reg_sports_share      0
apartment_build                0
apartment_fund_sqm             0
price_doc                      0
Length: 391, dtype: int64


<ipython-input-14-c4460d7ad137>:3: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  final_train_df.interpolate(method='linear',inplace=True)


In [ ]:
final_train_df.head()

if final_train_df.isnull().any().any():
    print("There is at least one null value in the DataFrame.")
else:
    print("There are no null values in the DataFrame.")

There is at least one null value in the DataFrame.


In [ ]:
# Find rows with null values

final_train_df.dropna(inplace=True, axis=1)
if final_train_df.isnull().any().any():
    print("There is at least one null value in the DataFrame.")
else:
    print("There are no null values in the DataFrame.")

final_train_df.tail()

There are no null values in the DataFrame.


,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm,price_doc
22410,30469,2015-06-30,44,27.0,7.0,9.0,1.0,1975.0,2.0,6.0,...,10805.0,627.0,0.45888,1440.0,463938.0,28.2,76.2,46080.0,234576.9,7400000
22411,30470,2015-06-30,86,59.0,3.0,9.0,2.0,1935.0,4.0,10.0,...,10805.0,627.0,0.45888,1440.0,463938.0,28.2,76.2,46080.0,234576.9,25000000
22412,30471,2015-06-30,45,45.5,10.0,20.0,1.0,1969.0,1.0,1.0,...,10805.0,627.0,0.45888,1440.0,463938.0,28.2,76.2,46080.0,234576.9,6970959
22413,30472,2015-06-30,64,32.0,5.0,15.0,1.0,2003.0,2.0,11.0,...,10805.0,627.0,0.45888,1440.0,463938.0,28.2,76.2,46080.0,234576.9,13500000
22414,30473,2015-06-30,43,28.0,1.0,9.0,1.0,1968.0,2.0,6.0,...,10805.0,627.0,0.45888,1440.0,463938.0,28.2,76.2,46080.0,234576.9,5600000


In [ ]:
# one hot encdoing for categorical columns

categorical_columns=final_train_df.select_dtypes(include=['object']).columns
final_train_df=pd.get_dummies(final_train_df,columns=categorical_columns)
final_train_df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory,"modern_education_share_90,92","modern_education_share_93,08","modern_education_share_95,4918","old_education_build_share_23,14","old_education_build_share_25,47","old_education_build_share_8,2517"
0,8059,2013-05-21,11,11.0,2.0,5.000000,2.000000,1907.000000,1.000000,12.000000,...,False,False,False,False,True,False,False,True,False,False
1,8060,2013-05-22,45,11.5,1.0,5.309091,2.036364,1907.924051,1.036364,11.781818,...,False,True,False,False,True,False,False,True,False,False
2,8061,2013-05-22,68,12.0,2.0,5.618182,2.072727,1908.848101,1.072727,11.563636,...,False,True,False,False,True,False,False,True,False,False
3,8062,2013-05-22,68,12.5,2.0,5.927273,2.109091,1909.772152,1.109091,11.345455,...,False,True,False,False,True,False,False,True,False,False
4,8063,2013-05-22,21,13.0,7.0,6.236364,2.145455,1910.696203,1.145455,11.127273,...,False,False,True,False,True,False,False,True,False,False


In [ ]:
final_train_df.dropna(inplace=True, axis=1)
if final_train_df.isnull().any().any():
    print("There is at least one null value in the DataFrame.")
else:
    print("There are no null values in the DataFrame.")

final_train_df.tail()

There are no null values in the DataFrame.


,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,ecology_good,ecology_no data,ecology_poor,ecology_satisfactory,"modern_education_share_90,92","modern_education_share_93,08","modern_education_share_95,4918","old_education_build_share_23,14","old_education_build_share_25,47","old_education_build_share_8,2517"
22410,30469,2015-06-30,44,27.0,7.0,9.0,1.0,1975.0,2.0,6.0,...,True,False,False,False,False,False,True,False,False,True
22411,30470,2015-06-30,86,59.0,3.0,9.0,2.0,1935.0,4.0,10.0,...,False,False,True,False,False,False,True,False,False,True
22412,30471,2015-06-30,45,45.5,10.0,20.0,1.0,1969.0,1.0,1.0,...,False,True,False,False,False,False,True,False,False,True
22413,30472,2015-06-30,64,32.0,5.0,15.0,1.0,2003.0,2.0,11.0,...,False,False,False,True,False,False,True,False,False,True
22414,30473,2015-06-30,43,28.0,1.0,9.0,1.0,1968.0,2.0,6.0,...,False,False,True,False,False,False,True,False,False,True


In [ ]:

# read the test data
test_df=pd.read_csv('/content/drive/MyDrive/machine_learning_dataset/test.csv')
test_df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,...,8,0,0,0,1,10,1,0,14,1
1,30475,2015-07-01,79.2,NaN,8,17,1,0.0,3,1.0,...,4,1,1,0,2,11,0,1,12,1
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,...,42,11,4,0,10,21,0,10,71,11
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,...,1,1,2,0,0,10,0,0,2,0
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,...,5,1,1,0,2,12,0,1,11,1


In [ ]:
#And check for null value in what columns

print(test_df.isnull().sum())
test_df['timestamp']=pd.to_datetime(test_df['timestamp'])
# train_data.set_index('timestamp',inplace=True)

id                       0
timestamp                0
full_sq                  0
life_sq               1176
floor                    0
                      ... 
church_count_5000        0
mosque_count_5000        0
leisure_count_5000       0
sport_count_5000         0
market_count_5000        0
Length: 291, dtype: int64


In [ ]:
# Handle missing values in the training dataset

test_df.interpolate(method='linear',inplace=True,)
test_df.isnull().sum()

<ipython-input-22-8e10d3bbfaa2>:3: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  test_df.interpolate(method='linear',inplace=True,)


,0
id,0
timestamp,0
full_sq,0
life_sq,0
floor,0
...,...
church_count_5000,0
mosque_count_5000,0
leisure_count_5000,0
sport_count_5000,0


In [ ]:
test_df.dropna(inplace=True, axis=1)

if test_df.isnull().any().any():
    print("There is at least one null value in the DataFrame.")
else:
    print("There are no null values in the DataFrame.")

There are no null values in the DataFrame.


In [ ]:
test_df.reset_index(drop=True,inplace=True)
test_df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,...,mosque_count_2000,leisure_count_2000,sport_count_2000,market_count_2000,green_part_3000,prom_part_3000,office_count_3000,office_sqm_3000,trc_count_3000,trc_sqm_3000,cafe_count_3000,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_count_3000_na_price,cafe_count_3000_price_500,cafe_count_3000_price_1000,cafe_count_3000_price_1500,cafe_count_3000_price_2500,cafe_count_3000_price_4000,cafe_count_3000_price_high,big_church_count_3000,church_count_3000,mosque_count_3000,leisure_count_3000,sport_count_3000,market_count_3000,green_part_5000,prom_part_5000,office_count_5000,office_sqm_5000,trc_count_5000,trc_sqm_5000,cafe_count_5000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000,cafe_count_5000_na_price,cafe_count_5000_price_500,cafe_count_5000_price_1000,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,3.0,Juzhnoe Butovo,2.615514e+07,178264,0.137846,0.041116,14080,11926.000000,11,14892,24750.000000,13,1,1,0,13,4,no,0,4,4,no,no,no,no,no,no,no,no,102618,47681,54937,30808,16251,14557,121369,59138,62231,26087,7410,...,1,0,5,0,14.69,2.87,0,0,3,73000,12,781.82,1227.27,1004.55,1,2,2,7,0,0,0,1,3,1,0,7,0,21.58,4.69,1,37550,8,299166,19,676.47,1088.24,882.35,2,5,4,8,0,0,0,1,10,1,0,14,1
1,30475,2015-07-01,79.2,22.9,8,17,1,0.0,3,1.0,1.0,Poselenie Vnukovskoe,2.553630e+07,4001,0.496315,0.007122,275,7079.000000,0,264,16358.000000,0,0,0,0,0,0,no,0,1,0,no,no,no,no,no,no,no,no,17790,8350,9443,574,297,277,2566,1356,1211,861,244,...,0,0,1,0,39.50,3.32,0,0,2,22000,10,680.00,1200.00,940.00,0,1,6,2,1,0,0,1,5,0,0,7,0,39.10,7.70,2,177300,6,231300,20,733.33,1250.00,991.67,2,4,8,4,1,1,0,2,11,0,1,12,1
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,2.0,Perovo,9.946335e+06,139322,0.065409,0.225825,6400,2232.000000,7,6558,7966.000000,7,0,3,0,13,0,no,0,2,7,yes,no,yes,yes,no,no,no,no,36154,16222,19932,13799,6937,6862,91795,44734,47061,33728,9653,...,0,4,14,4,45.86,9.08,8,215237,6,39106,37,608.33,1069.44,838.89,1,8,19,7,2,0,0,2,3,0,5,22,4,25.62,13.59,27,427889,26,1024431,179,668.97,1132.18,900.57,5,53,64,42,11,4,0,10,21,0,10,71,11
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,3.0,Poselenie Voskresenskoe,2.149409e+07,7122,0.262459,0.017647,489,1995.333333,0,469,6945.333333,0,0,0,0,0,2,no,0,0,0,no,no,no,no,no,no,no,no,9553,4529,5024,1021,529,493,4568,2414,2155,1533,435,...,0,0,0,0,34.87,1.34,0,0,0,0,1,300.00,500.00,400.00,0,1,0,0,0,0,0,0,4,0,0,0,0,24.25,1.66,0,0,0,0,5,1560.00,2500.00,2030.00,0,1,0,1,1,2,0,0,10,0,0,2,0
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,1.0,Poselenie Vnukovskoe,2.553630e+07,4001,0.496315,0.007122,275,1758.666667,0,264,5924.666667,0,0,0,0,0,0,no,0,1,0,no,no,no,no,no,no,no,no,17790,8350,9443,574,297,277,2566,1356,1211,861,244,...,0,0,3,0,41.64,2.11,0,0,2,22000,9,700.00,1222.22,961.11,0,1,5,2,1,0,0,1,4,0,0,6,0,35.62,6.96,1,117300,4,201300,20,747.37,1263.16,1005.26,1,4,8,5,1,1,0,2,12,0,1,11,1


In [ ]:
#Add macro file with timestamp

from typing_extensions import final
#perform the join

final_test_df=pd.merge(test_df, macro_data,on='timestamp', how='left')
final_test_df.tail()


,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,...,invest_fixed_assets_phys,pop_natural_increase,pop_migration,pop_total_inc,childbirth,mortality,housing_fund_sqm,lodging_sqm_per_cap,water_pipes_share,baths_share,sewerage_share,gas_share,hot_water_share,electric_stove_share,heating_share,old_house_share,average_life_exp,infant_mortarity_per_1000_cap,perinatal_mort_per_1000_cap,incidence_population,rent_price_4+room_bus,rent_price_3room_bus,rent_price_2room_bus,rent_price_1room_bus,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,load_of_teachers_preschool_per_teacher,child_on_acc_pre_school,load_of_teachers_school_per_teacher,students_state_oneshift,modern_education_share,old_education_build_share,provision_doctors,provision_nurse,load_on_doctors,power_clinics,hospital_beds_available_per_cap,hospital_bed_occupancy_per_year,provision_retail_space_sqm,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
7657,38131,2016-05-26,52.20,31.8,10,12,5,1973.0,2,9.1,2.0,Kon'kovo,7.811375e+06,153248,0.127002,0.135650,7517,2627.0,5,7960,9650.0,5,0,3,0,11,3,no,0,11,6,no,no,no,yes,no,no,no,no,123000,58226,64774,16468,8851,7617,100062,55456,44606,36718,11958,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.89,99.88,73.46,50.11,47.57,36.96,29.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7658,38132,2016-05-28,54.09,16.4,14,0,1,987.0,2,0.0,1.5,Poselenie Desjonovskoe,5.299528e+07,13890,0.349935,0.011654,953,2250.5,0,915,8211.0,0,0,0,0,0,1,no,0,0,0,no,no,no,no,no,no,no,no,21819,10343,11477,1991,1030,962,8910,4707,4203,2989,847,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.89,99.88,73.46,50.11,47.57,36.96,29.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7659,38133,2016-05-30,41.08,1.0,12,1,1,1.0,1,1.0,1.0,Tverskoe,7.307411e+06,75377,0.065444,0.000078,4237,1874.0,4,6398,6772.0,4,1,3,2,29,16,yes,10,23,141,no,no,no,yes,yes,no,no,yes,116742,52836,63906,11272,5470,5802,43921,21901,22020,20184,6644,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.89,99.88,73.46,50.11,47.57,36.96,29.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7660,38134,2016-05-30,34.80,19.8,8,9,5,1977.0,1,6.4,2.0,Orehovo-Borisovo Juzhnoe,7.128794e+06,145576,0.075790,0.100456,5594,4792.0,7,7227,11178.0,7,1,1,0,7,2,no,0,6,0,no,no,no,no,no,yes,no,no,129207,55959,73248,13595,6998,6597,104635,50319,54316,27346,8321,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.89,99.88,73.46,50.11,47.57,36.96,29.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7661,38135,2016-05-30,63.00,43.8,5,5,1,1973.0,3,7.1,3.0,Chertanovo Severnoe,6.206099e+06,111874,0.128123,0.089040,4720,2881.0,5,7976,6784.0,5,0,0,0,10,3,no,0,11,5,no,no,no,no,no,no,no,no,125354,56569,68785,13326,5986,7340,73503,34020,39483,25045,6881,...,NaN,NaN,NaN,NaN,N

In [ ]:
# hanle misinf value

final_test_df.interpolate(method='linear',inplace=True)
print(final_test_df.isnull().sum())

id                                0
timestamp                         0
full_sq                           0
life_sq                           0
floor                             0
                               ... 
bandwidth_sports                  0
population_reg_sports_share    7662
students_reg_sports_share      7662
apartment_build                7662
apartment_fund_sqm                0
Length: 388, dtype: int64


<ipython-input-26-48166297883d>:3: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  final_test_df.interpolate(method='linear',inplace=True)


In [ ]:
# Find rows with null values

final_test_df.dropna(inplace=True, axis=1)
if final_test_df.isnull().any().any():
    print("There is at least one null value in the DataFrame.")
else:
    print("There are no null values in the DataFrame.")

final_test_df.tail()

There are no null values in the DataFrame.


,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,...,balance_trade_growth,usdrub,eurrub,brent,net_capital_export,gdp_annual,gdp_annual_growth,average_provision_of_build_contract,average_provision_of_build_contract_moscow,rts,micex,micex_rgbi_tr,micex_cbi_tr,deposits_value,deposits_growth,deposits_rate,mortgage_value,mortgage_growth,mortgage_rate,income_per_cap,salary,salary_growth,fixed_basket,retail_trade_turnover,retail_trade_turnover_per_cap,retail_trade_turnover_growth,labor_force,unemployment,employment,invest_fixed_capital_per_cap,invest_fixed_assets,pop_natural_increase,childbirth,mortality,average_life_exp,rent_price_4+room_bus,rent_price_3room_bus,rent_price_2room_bus,rent_price_1room_bus,rent_price_3room_eco,rent_price_2room_eco,rent_price_1room_eco,load_of_teachers_school_per_teacher,students_state_oneshift,provision_nurse,load_on_doctors,turnover_catering_per_cap,seats_theather_rfmin_per_100000_cap,bandwidth_sports,apartment_fund_sqm
7657,38131,2016-05-26,52.20,31.8,10,12,5,1973.0,2,9.1,2.0,Kon'kovo,7.811375e+06,153248,0.127002,0.135650,7517,2627.0,5,7960,9650.0,5,0,3,0,11,3,no,0,11,6,no,no,no,yes,no,no,no,no,123000,58226,64774,16468,8851,7617,100062,55456,44606,36718,11958,...,8.0,65.5700,73.2735,49.59,-0.523202,80804.3,-0.037267,5.94,6.22,918.79,1914.31,130.66,301.04,22673775,0.006879,7.04,446114,0.497746,12.94,102709.0,64310.0,0.05068,20031.98,4310.1067,351.447778,82.8,7067.470079,0.017708,0.733598,131403.0,1611.511527,1.66,11.55,9.89,76.77,205.89,99.88,73.46,50.11,47.57,36.96,29.16,1573.650623,97.8484,90.8,6899.930464,10805.0,0.45888,463938.0,234576.9
7658,38132,2016-05-28,54.09,16.4,14,0,1,987.0,2,0.0,1.5,Poselenie Desjonovskoe,5.299528e+07,13890,0.349935,0.011654,953,2250.5,0,915,8211.0,0,0,0,0,0,1,no,0,0,0,no,no,no,no,no,no,no,no,21819,10343,11477,1991,1030,962,8910,4707,4203,2989,847,...,8.0,65.9385,73.2557,49.32,-0.523202,80804.3,-0.037267,5.94,6.22,917.52,1927.58,130.55,301.67,22673775,0.006879,7.04,446114,0.497746,12.94,102709.0,64310.0,0.05068,20031.98,4310.1067,351.447778,82.8,7067.470079,0.017708,0.733598,131403.0,1611.511527,1.66,11.55,9.89,76.77,205.89,99.88,73.46,50.11,47.57,36.96,29.16,1573.650623,97.8484,90.8,6899.930464,10805.0,0.45888,463938.0,234576.9
7659,38133,2016-05-30,41.08,1.0,12,1,1,1.0,1,1.0,1.0,Tverskoe,7.307411e+06,75377,0.065444,0.000078,4237,1874.0,4,6398,6772.0,4,1,3,2,29,16,yes,10,23,141,no,no,no,yes,yes,no,no,yes,116742,52836,63906,11272,5470,5802,43921,21901,22020,20184,6644,...,8.0,65.6745,73.1241,49.76,-0.523202,80804.3,-0.037267,5.94,6.22,925.51,1935.54,130.57,301.73,22673775,0.006879,7.04,446114,0.497746,12.94,102709.0,64310.0,0.05068,20031.98,4310.1067,351.447778,82.8,7067.470079,0.017708,0.733598,131403.0,1611.511527,1.66,11.55,9.89,76.77,205.89,99.88,73.46,50.11,47.57,36.96,29.16,1573.650623,97.8484,90.8,6899.930464,10805.0,0.45888,463938.0,234576.9
7660,38134,2016-05-30,34.80,19.8,8,9,5,1977.0,1,6.4,2.0,Orehovo-Borisovo Juzhnoe,7.128794e+06,145576,0.075790,0.100456,5594,4792.0,7,7227,11178.0,7,1,1,0,7,2,no,0,6,0,no,no,no,no,no,yes,no,no,129207,55959,73248,13595,6998,6597,104635,50319,54316,27346,8321,...,8.0,65.6745,73.1241,49.76,-0.523202,80804.3,-0.037267,5.94,6.22,925.51,1935.54,130.57,301.73,22673775,0.006879,7.04,446114,0.497746,12.94,102709.0,64310.0

In [ ]:
for col in final_test_df.select_dtypes(include=['datetime64']).columns:
    # You might need to adjust the unit based on your data (e.g., 's' for seconds, 'D' for days)
    final_test_df[col] = final_test_df[col].astype(int)


In [ ]:
# Now differentiate the the target variable

for col in final_train_df.select_dtypes(include=['datetime64']).columns:
    # You might need to adjust the unit based on your data (e.g., 's' for seconds, 'D' for days)
    final_train_df[col] = final_train_df[col].astype(int)


# Reduce the sample size for training data
sample_size = int(0.2 * len(final_train_df))  # 10% of the data
resample_train_df = final_train_df.sample(n=sample_size, random_state=42)

# Get the columns present in both dataframes
common_cols = final_train_df.columns.intersection(final_test_df.columns)
print(common_cols)

X=resample_train_df[common_cols]
y=resample_train_df['price_doc']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

x_train.dropna()

print(x_train.shape)
print(y_train.shape)


# Assuming x_train and y_train are prepared from final_train_df
# Ensure that final_test_df and final_train_df have the same columns before proceeding.


# Select only these common columns for both training and testing
x_test = final_test_df[common_cols]

Index(['id', 'timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor',
       'material', 'build_year', 'num_room', 'kitch_sq',
       ...
       'rent_price_2room_eco', 'rent_price_1room_eco',
       'load_of_teachers_school_per_teacher', 'students_state_oneshift',
       'provision_nurse', 'load_on_doctors', 'turnover_catering_per_cap',
       'seats_theather_rfmin_per_100000_cap', 'bandwidth_sports',
       'apartment_fund_sqm'],
      dtype='object', length=332)
(3586, 332)
(3586,)


In [ ]:
# !pip install scikit-learn
# from sklearn.neural_network import MLPRegressor
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.svm import SVR
# from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score

MLPRegressor_param = {
        'hidden_layer_sizes': [(50,50), (100,),(50,)],
        'activation': ['relu', 'tanh'],
        'solver': ['adam', 'sgd'],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate': ['constant','adaptive']}

RandomForestRegressor_param = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt']
        }

DecisionTreeRegressor_param = {
        'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
        'splitter': ['best', 'random'],
        'max_depth': [None, 5, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt']
        }

# SVR_param = {
#         'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
#         'degree': [2, 3, 4],  # Only for poly kernel
#         'gamma': ['scale', 'auto'],
#         'C': [0.1, 1, 10],
#         'epsilon': [0.01, 0.1, 0.2]
#         }

SVR_param = {
    'epsilon': [0, 0.1, 0.2],
    'C': [0.1, 1, 10],
    'loss': ['epsilon_insensitive', 'squared_epsilon_insensitive'],
    'dual': [True, False],
    'fit_intercept': [True, False],
    'max_iter': [20000, 300000]
}

GradientBoostingRegressor_param = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'loss': ['squared_error', 'absolute_error', 'huber']
        }

regressor_model={

    "MLPRegressor": MLPRegressor(max_iter=500, early_stopping=True, validation_fraction=0.1),
    "RandomForestRegressor": RandomForestRegressor(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    # "linear SVR": LinearSVR(),
    # "GradientBoostingRegressor": GradientBoostingRegressor()

    }

for model_name, model in regressor_model.items():
      print(f"Training {model_name}...")
      print(model)

      # different parameter
      if model_name == "MLPRegressor":
          parameters = MLPRegressor_param
      elif model_name == "RandomForestRegressor":
          parameters = RandomForestRegressor_param
      elif model_name == "DecisionTreeRegressor":
          parameters = DecisionTreeRegressor_param
      elif model_name == "linear SVR":
          parameters = SVR_param
      elif model_name == "GradientBoostingRegressor":
          parameters = GradientBoostingRegressor_param
      else:
          parameters = None

      # fit the model
      model.fit(x_train, y_train)

      # Hypertuning using RandomSearch
      if parameters:
        regressor_random = RandomizedSearchCV(model,parameters,n_iter=15)
        regressor_random.fit(x_train, y_train)
        reg_best_param=regressor_random.best_params_
        reg_best_cv_score = cross_val_score(regressor_random.best_estimator_, x_train, y_train, cv=5).mean()
        final_reg_model=model.__class__(**reg_best_param)
      else:
        print("No parameters available for this model.")


      final_reg_model.fit(x_train, y_train) # Fit the model here
      y_pred=final_reg_model.predict(x_test)

      print("Best Parameters (Random Search):", reg_best_param)
      print("Cross-Validated Accuracy (Random Search):", reg_best_cv_score)
      y_pred=final_reg_model.predict(x_test)
      # create a data frame to store Y_pred

      pred_df=pd.DataFrame(y_pred, columns=['price_doc'])

      # concat the data frame
      final_pred_df=pd.concat([final_test_df['id'], pred_df], axis=1)
      final_pred_df.to_csv(f"{model_name}_submission.csv", index=False)


Training MLPRegressor...
MLPRegressor(early_stopping=True, max_iter=500)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Best Parameters (Random Search): {'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (100,), 'alpha': 0.001, 'activation': 'tanh'}
Cross-Validated Accuracy (Random Search): -0.0002261321869742261
Training RandomForestRegressor...
RandomForestRegressor()


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
40 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_

Best Parameters (Random Search): {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10}
Cross-Validated Accuracy (Random Search): 0.567511559891869
Training DecisionTreeRegressor...
DecisionTreeRegressor()


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
30 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_

Best Parameters (Random Search): {'splitter': 'best', 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 5, 'criterion': 'squared_error'}
Cross-Validated Accuracy (Random Search): 0.2180197589274809


In [ ]:
# new dataset for stacking

x1_train=final_train_df[common_cols]
y1_train=final_train_df['price_doc']
#Stack the model

model=[ MLPRegressor(max_iter=500, early_stopping=True, validation_fraction=0.1), RandomForestRegressor(), DecisionTreeRegressor()]
# Ensemble Stacking and stacked model

S_train, S_test= stacking(
    model,x1_train, y1_train,x_test,
    regression=True,
    mode='oof_pred_bag',
    needs_proba=False,
    save_dir=None,
    n_folds=4,
    stratified=False,
    shuffle=True,
    random_state=0,
    verbose=2)

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [MLPRegressor]
    fold  0:  [2925500.10379876]
    fold  1:  [3232209.34312566]
    fold  2:  [5881918.24178260]
    fold  3:  [6109637.61018578]
    ----
    MEAN:     [4537316.32472320] + [1464701.70201378]
    FULL:     [4537246.17878140]

model  1:     [RandomForestRegressor]
    fold  0:  [1374222.11240364]
    fold  1:  [1405105.96091006]
    fold  2:  [1358361.41208423]
    fold  3:  [1399586.22823844]
    ----
    MEAN:     [1384318.92840909] + [18979.78866768]
    FULL:     [1384318.24728932]

model  2:     [DecisionTreeRegressor]
    fold  0:  [1935187.16630978]
    fold  1:  [1923164.42897930]
    fold  2:  [1861442.58654532]
    fold  3:  [1978690.81813314]
    ----
    MEAN:     [1924621.24999189] + [41919.03693125]
    FULL:     [1924618.83778720]



In [ ]:
#Best stack model

param = {
        'n_estimators': [200, 300, 400],
        'max_depth': [None, 15, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt']
        }

# Create a stack model

stack_model = RandomForestRegressor()
stack_model.fit(S_train, y1_train)

RandomForestRegressor()

In [ ]:
# Using Hypertuning and cross-validation

stack_random = RandomizedSearchCV(stack_model,param,n_iter=15)
stack_random.fit(S_train, y1_train)
stk_best_param=stack_random.best_params_
stk_best_cv_score = cross_val_score(stack_random.best_estimator_, S_train, y1_train, cv=5).mean()
final_stk_model=stack_random.best_estimator_.__class__(**stk_best_param)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
45 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_

In [ ]:
# Stack the final model and predict the target variable

final_stk_model.fit(S_train, y1_train)
Y_pred=final_stk_model.predict(S_test)

In [ ]:
# convert the y_pred to df and then concat it and save it to csv

print("Best Parameters (Random Search):", stk_best_param)
print("Cross-Validated Accuracy (Random Search):", stk_best_cv_score)

# create a data frame to store Y_pred

pred_df1=pd.DataFrame(Y_pred, columns=['price_doc'])

# concat the data frame
final_pred_df_stk=pd.concat([final_test_df['id'], pred_df1], axis=1)
final_pred_df_stk.to_csv(f"{stack_model}__stack_submission.csv", index=False)

Best Parameters (Random Search): {'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 15}
Cross-Validated Accuracy (Random Search): 0.6767990545368009


# Now do feature selection using various methods and then predict using thiose features



In [ ]:
# load the data-set for feature selection

x_train , y_train

#Step 2: Train a model and use SelectFromModel
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
# Select features based on feature importance
sfm = SelectFromModel(estimator=model, threshold="mean")
sfm.fit(x_train, y_train)

selected_features = x_train.columns[sfm.get_support()]
print(f"Selected Features: {list(selected_features)}")

# new data set for selected feature

x_train_selected = x_train[selected_features]
x_test_selected = x_test[selected_features]

# train new model with new selected feature'

feature_model=RandomForestRegressor(n_estimators=100, random_state=42)
feature_model.fit(x_train_selected, y_train)

Selected Features: ['full_sq', 'life_sq', 'floor', 'max_floor', 'kitch_sq', 'state', 'kindergarten_km', 'water_km', 'ttk_km', 'sadovoe_km', 'bulvar_ring_km', 'zd_vokzaly_avto_km', 'nuclear_reactor_km', 'stadium_km', 'public_healthcare_km', 'office_km', 'additional_education_km', 'big_church_km', 'theater_km', 'museum_km', 'cafe_count_1000', 'cafe_count_1500_price_1500', 'office_sqm_2000', 'trc_count_2000', 'trc_sqm_2000', 'cafe_count_2000', 'cafe_count_2000_price_1500', 'office_count_3000', 'trc_sqm_3000', 'cafe_count_3000', 'cafe_count_3000_price_500', 'cafe_count_3000_price_1000', 'cafe_count_3000_price_1500', 'cafe_count_3000_price_2500', 'sport_count_3000', 'office_sqm_5000', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500', 'cafe_count_5000_price_high', 'sport_count_5000', 'micex', 'micex_cbi_tr', 'rent_price_4+room_bus']


RandomForestRegressor(random_state=42)

In [ ]:
# predict the model
y_pred_selected = feature_model.predict(x_test_selected)

# save file to data frame
Y_feature_P_df=pd.DataFrame(y_pred_selected, columns=['price_doc'])

# concact the data frame to create new submission file.

final_fearture_pred_df=pd.concat([final_test_df['id'], Y_feature_P_df], axis=1)

# save to csv
final_fearture_pred_df.to_csv(f"{feature_model}__feature_selection_submission.csv", index=False)